In [1]:
MY_API_KEY = "ZM85KK6A7M4CG1NA7PJYP6KZVDE9"

In [2]:
import sys
# Delete the above line, and uncomment the below line if you are using conda
# !conda install --yes --prefix {sys.prefix} numpy

import requests
import pprint
import json

Searching

In [ ]:
url = "https://beta4.api.climatiq.io/search"
query="grid mix"

query_params = {
    # Free text query can be writen under the query string
    "query": query,
    # You can also filter on region, year, source and more
    "region": "AT",
    # The Climatiq emission factors are versioned.
    # The string below means "major version 1 compatible", which means we stay on major version 1 of the data
    # but get new emission factors as they are available
    # Check out the documentation for more information about data versioning
    "data_version": "^1",
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}


search_response = requests.get(url, params=query_params, headers=authorization_headers).json()
# The search endpoint returns a lot of stuff - here are the keys
print(search_response.keys())

# The most relevant is probably the results - here are the first three results
search_response["results"][0:3]

Tabular Format Changes

In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.json_normalize(search_response)

,current_page,last_page,total_results,results,possible_filters.year,possible_filters.source,possible_filters.region,possible_filters.category,possible_filters.sector,possible_filters.unit_type,possible_filters.source_lca_activity,possible_filters.access_type,possible_filters.data_quality_flags
0,1,3,49,[{'activity_id': 'electricity-supply_grid-sour...,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...","[{'source': 'ADEME', 'datasets': ['Base Carbon...","[{'id': 'AT', 'name': 'Austria'}]",[Electricity],[Energy],[Energy],"[electricity_generation, well_to_tank]",[public],"[partial_factor, notable_methodological_variance]"


Estimates

In [7]:
# The id for the emission factor. Found via the data explorer or the search endpoint
# In our case, we take the first activity ID from our search above
activity_id = search_response["results"][0]["activity_id"]
# We have many regions with the same activity id, representing the power grid in different countries.
# We'd like to get the power for Australia specifically, so we will need to specify a region. We do this by specifying the UN location code for the region
# You can also see the region for different emission factors in the data explorer
region = "AT"

# We must also specify how much electricity generation we would like to make the estimation for.
# In this case we will do it for 100kilowattt-hours. 
# Different emission factors have different requirement as to what units they support estimates for
# You can see the units supported by an emission factor in the data explorer, and find more documentation about units
# in the API documentation.
parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

json_body = {
    "emission_factor": {
        "activity_id": activity_id,
        "region": region,
        "data_version": "^1"
        # More filters are possible here. See the full documentation for more options
    },
    # Specify how much energy we're estimating for
    "parameters": parameters
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the estimate endpoint with a json body and the correct authorization headers
response = requests.post("https://beta4.api.climatiq.io/estimate", json=json_body, headers=authorization_headers)
# The response is also json.
pprint.pprint(response.json())


{'activity_data': {'activity_unit': 'kWh', 'activity_value': 100.0},
 'audit_trail': 'selector',
 'co2e': 54.586999999999996,
 'co2e_calculation_method': 'ar5',
 'co2e_calculation_origin': 'source',
 'co2e_unit': 'kg',
 'constituent_gases': {'ch4': None,
                       'co2': 54.586999999999996,
                       'co2e_other': None,
                       'co2e_total': 54.586999999999996,
                       'n2o': None},
 'emission_factor': {'access_type': 'public',
                     'activity_id': 'electricity-supply_grid-source_residual_mix',
                     'category': 'Electricity',
                     'data_quality_flags': ['partial_factor',
                                            'notable_methodological_variance'],
                     'id': '205cf321-ee05-46d3-aa20-b2046d96ab31',
                     'name': 'Electricity supplied from grid - residual mix',
                     'region': 'AT',
                     'source': 'GHG Protocol',
          

Tabular Format Changes

In [8]:
import pandas as pd
pd.json_normalize(search_response)

,current_page,last_page,total_results,results,possible_filters.year,possible_filters.source,possible_filters.region,possible_filters.category,possible_filters.sector,possible_filters.unit_type,possible_filters.source_lca_activity,possible_filters.access_type,possible_filters.data_quality_flags
0,1,3,49,[{'activity_id': 'electricity-supply_grid-sour...,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...","[{'source': 'ADEME', 'datasets': ['Base Carbon...","[{'id': 'AT', 'name': 'Austria'}]",[Electricity],[Energy],[Energy],"[electricity_generation, well_to_tank]",[public],"[partial_factor, notable_methodological_variance]"


Batch Estimate

In [ ]:
#activity_id = 
#region = 

In [ ]:
parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

# Notice that the JSON body here is an array, with each entry being an estimate
json_body = [
{
    "emission_factor": {
        # Using the same activity ID and region as before
        "activity_id": activity_id,
        "region": region,
        "data_version": "^1"
    },
    "parameters": {
        "energy": 10,
        "energy_unit": "kWh"
    }
},
{
    "emission_factor": {
        # Using the same activity ID and region as before
        "activity_id": activity_id,
        "region": region,
        "data_version": "^1"
    },
    "parameters": {
        "energy": 100,
        "energy_unit": "kWh"
    }
},
{
    "emission_factor": {
        # Using the same activity ID and region as before
        "activity_id": activity_id,
        "region": region,
        "data_version": "^1"
    },
    "parameters": {
        "energy": 100,
        "energy_unit": "kWh"
    }
},
{
    "emission_factor": {
        # Using the same activity ID and region as before
        "activity_id": activity_id,
        "region": region,
        "data_version": "^1"
    },
    "parameters": {
        "energy": 100,
        "energy_unit": "kWh"
    }
},
{
    "emission_factor": {
        # Using the same activity ID and region as before
        "activity_id": activity_id,
        "region": region,
        "data_version": "^1"
    },
    "parameters": {
        "energy": 100,
        "energy_unit": "kWh"
    }
}
]


# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the /batch endpoint with a json body and the correct authorization headers
response = requests.post("https://beta4.api.climatiq.io/batch", json=json_body, headers=authorization_headers)
# The response is also json.
response_json = response.json()
response_json

Tabular Format Changes

In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)
normalized_data = pd.json_normalize(response_json['results'])
normalized_data

pd.to_datetime('now')

NameError: name 'response_json' is not defined